This notebook has information about Data extraction from github repo and storing it in mongodb

In [1]:
from bs4 import BeautifulSoup
# from pytube import YouTube
from pymongo import MongoClient
from clearml import Task

import os
import shutil
import subprocess
import tempfile

In [113]:
def connect_mongodb():
    """Connect to MongoDB."""
    client = MongoClient("mongodb://localhost:27017/")
    return client["RAG_DB"], client["RAG_DB"]["raw_data"]


In [3]:
def extract_github(repo_url):
        """Extract content from a GitHub repository."""

        repo_name = repo_url.rstrip("/").split("/")[-1]
        temp_dir = tempfile.mkdtemp()

        try:
            # Clone the repository into a temporary directory
            os.chdir(temp_dir)
            subprocess.run(["git", "clone", repo_url], check=True)

            # Determine the cloned repository's path
            repo_path = os.path.join(temp_dir, os.listdir(temp_dir)[0])

            # Parse files in the repository
            tree = {}
            for root, _, files in os.walk(repo_path):
                dir = root.replace(repo_path, "").lstrip("/")
                if dir.startswith((".git", ".toml", ".lock", ".png")):
                    continue

                for file in files:
                    if file.endswith((".git", ".toml", ".lock", ".png")):
                        continue
                    file_path = os.path.join(dir, file)
                    with open(os.path.join(root, file), "r", errors="ignore") as f:
                        tree[file_path] = f.read()

            data_entry = {
                "content": tree,
                "url": repo_url,
                "source": "github",
                "repo_name": repo_name,
            }

            # self.ingested_urls.append(repo_url)
            return data_entry

        finally:
            # Cleanup temporary directory
            shutil.rmtree(temp_dir)

In [21]:
github_data = extract_github("https://github.com/ros2/ros2_documentation")

Cloning into 'ros2_documentation'...


In [10]:
github_data.keys()

dict_keys(['content', 'url', 'source', 'repo_name'])

These are all the files in the github repository

In [86]:
# These are all the files in the github repo:
files = github_data["content"].keys()
list(files)

['.mergify.yml',
 'constraints.txt',
 'favicon.ico',
 'LICENSE',
 'requirements.txt',
 'app.json',
 'Makefile',
 'CODEOWNERS',
 'conf.py',
 'make_sitemapindex.py',
 'rosindex.yml',
 'README.md',
 'global_substitutions.txt',
 'setup.py',
 '.gitignore',
 'docker/image/Dockerfile',
 'plugins/sphinx_sitemap_ros.py',
 'source/index.rst',
 'source/Package-Docs.rst',
 'source/Concepts.rst',
 'source/Related-Projects.rst',
 'source/How-To-Guides.rst',
 'source/Contact.rst',
 'source/The-ROS2-Project.rst',
 'source/Citations.rst',
 'source/Installation.rst',
 'source/Releases.rst',
 'source/Tutorials.rst',
 'source/Glossary.rst',
 'source/The-ROS2-Project/Roadmap.rst',
 'source/The-ROS2-Project/Contributing.rst',
 'source/The-ROS2-Project/Features.rst',
 'source/The-ROS2-Project/Governance.rst',
 'source/The-ROS2-Project/Marketing.rst',
 'source/The-ROS2-Project/Metrics.rst',
 'source/The-ROS2-Project/Feature-Ideas.rst',
 'source/The-ROS2-Project/ROSCon-Content.rst',
 'source/The-ROS2-Project/C

In [87]:
def clean_text(raw_text):
    """Clean and process raw text data."""
    soup = BeautifulSoup(raw_text, "html.parser")
    return soup.get_text().strip()
import re

In [79]:
def clean_data(content):
    """Clean raw content for better transformer embeddings."""
    # Remove code blocks (fenced with triple backticks)
    content = re.sub(r'```.*?```', '', content, flags=re.DOTALL)
    
    # Remove markdown artifacts like #, *, -, `, and excessive newlines
    content = re.sub(r'[\n#`*\\-]', ' ', content)
    
    # Flatten lists (e.g., "- item" to "item")
    content = re.sub(r'^\s*-\s*', '', content, flags=re.MULTILINE)
    
    # Replace multiple spaces with a single space
    content = re.sub(r'\s+', ' ', content).strip()
    
    # Normalize text (e.g., convert to lowercase)
    content = content.lower()
    
    return content

1. Then we will clean all the text files
2.  Store the code files
3. Ignore other files

In [103]:
def transform_data(data):
    """Clean and transform the extracted data."""
    if not data or not data.get("content"):
        print("No content to transform.")
        return None
    else:
        transformed_data = {}
        for file, content in data["content"].items():
            if file.endswith((".md", ".rst", ".txt")):
                transformed_data[file] = {
                    "content": clean_data(clean_text(content)),
                    "file_name": file,
                    "url": data["url"],
                    "source": data["source"],
                    "repo_name": data["repo_name"],
                }
            elif file.endswith((".py", ".cpp", ".c", ".ipynb")):
                transformed_data[file] = {
                    "content": content,
                    "file_name": file,
                    "url": data["url"],
                    "source": data["source"],
                    "repo_name": data["repo_name"],
                }
        return transformed_data

In [104]:
transformed_data = transform_data(github_data)

/Users/anshharjai/anaconda3/envs/ai_project/lib/python3.11/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [ ]:
# cleaned_data
print(transformed_data['constraints.txt']['content'])

alabaster==0.7.12 babel==2.14.0 certifi==2020.6.20 chardet==4.0.0 doc8==1.1.1 docutils==0.20.1 idna==2.10 imagesize==1.3.0 jinja2==3.0.3 markupsafe==2.0.1 packaging==21.3 pbr==5.8.0 polib==1.2.0 pygments==2.17.2 pyparsing==2.4.7 pytz==2022.1 requests==2.25.1 restructuredtext_lint==1.3.2 snowballstemmer==2.2.0 sphinx==7.2.6 sphinx copybutton==0.5.2 sphinx lint==0.9.1 sphinx multiversion==0.2.4 sphinx rtd theme==2.0.0 sphinx tabs==3.4.5 sphinxcontrib applehelp==1.0.4 sphinxcontrib devhelp==1.0.2 sphinxcontrib htmlhelp==2.0.1 sphinxcontrib jquery==4.1 sphinxcontrib jsmath==1.0.1 sphinxcontrib mermaid==0.9.2 sphinxcontrib qthelp==1.0.3 sphinxcontrib serializinghtml==1.1.10 stevedore==3.5.0 urllib3==1.26.5


In [91]:
# original data
print(github_data["content"]["constraints.txt"])

alabaster==0.7.12
Babel==2.14.0
certifi==2020.6.20
chardet==4.0.0
doc8==1.1.1
docutils==0.20.1
idna==2.10
imagesize==1.3.0
Jinja2==3.0.3
MarkupSafe==2.0.1
packaging==21.3
pbr==5.8.0
polib==1.2.0
Pygments==2.17.2
pyparsing==2.4.7
pytz==2022.1
requests==2.25.1
restructuredtext_lint==1.3.2
snowballstemmer==2.2.0
Sphinx==7.2.6
sphinx-copybutton==0.5.2
sphinx-lint==0.9.1
sphinx-multiversion==0.2.4
sphinx-rtd-theme==2.0.0
sphinx-tabs==3.4.5
sphinxcontrib-applehelp==1.0.4
sphinxcontrib-devhelp==1.0.2
sphinxcontrib-htmlhelp==2.0.1
sphinxcontrib-jquery==4.1
sphinxcontrib-jsmath==1.0.1
sphinxcontrib-mermaid==0.9.2
sphinxcontrib-qthelp==1.0.3
sphinxcontrib-serializinghtml==1.1.10
stevedore==3.5.0
urllib3==1.26.5



In [114]:
def load_to_mongodb(data, db, collection):
    """Load transformed data into MongoDB."""
    if not data:
        print("No data to load.")
        return

    for file_name, file_data in data.items():
        print(f"Loading {file_name} into MongoDB.")
        collection.update_one(
            {"file_name": file_name},
            {"$set": file_data},
            upsert=True
        )
    print("Data loaded into MongoDB.")

# Load the transformed data into MongoDB
db, collection = connect_mongodb()
load_to_mongodb(transformed_data, db, collection)


Loading constraints.txt into MongoDB.
Loading requirements.txt into MongoDB.
Loading conf.py into MongoDB.
Loading make_sitemapindex.py into MongoDB.
Loading README.md into MongoDB.
Loading global_substitutions.txt into MongoDB.
Loading setup.py into MongoDB.
Loading plugins/sphinx_sitemap_ros.py into MongoDB.
Loading source/index.rst into MongoDB.
Loading source/Package-Docs.rst into MongoDB.
Loading source/Concepts.rst into MongoDB.
Loading source/Related-Projects.rst into MongoDB.
Loading source/How-To-Guides.rst into MongoDB.
Loading source/Contact.rst into MongoDB.
Loading source/The-ROS2-Project.rst into MongoDB.
Loading source/Citations.rst into MongoDB.
Loading source/Installation.rst into MongoDB.
Loading source/Releases.rst into MongoDB.
Loading source/Tutorials.rst into MongoDB.
Loading source/Glossary.rst into MongoDB.
Loading source/The-ROS2-Project/Roadmap.rst into MongoDB.
Loading source/The-ROS2-Project/Contributing.rst into MongoDB.
Loading source/The-ROS2-Project/Feat

In [93]:
def run_etl():
    """Run the ETL pipeline."""
    db, collection = connect_mongodb()
    if db is None or collection is None:
        print("MongoDB connection failed.")

    github_data = extract_github("https://github.com/ros2/ros2_documentation")
    
    transformed_data = transform_data(github_data)
    print("Transformed GitHub Data:\n", transformed_data)
    
    # Load
    load_to_mongodb(transformed_data, db, collection)
